In [1]:
import pandas as pd
df=pd.read_csv("ready-to-use-titanic.csv")
df

,Unnamed: 0,Survived,Age,firstclass,secondclass,thirdclass,C,Q,S,female,male
0,0,0,22.0,0,0,1,0,0,1,0,1
1,1,1,38.0,1,0,0,1,0,0,1,0
2,2,1,26.0,0,0,1,0,0,1,1,0
3,3,1,35.0,1,0,0,0,0,1,1,0
4,4,0,35.0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
709,709,0,39.0,0,0,1,0,1,0,1,0
710,710,0,27.0,0,1,0,0,0,1,0,1
711,711,1,19.0,1,0,0,0,0,1,1,0
712,712,1,26.0,1,0,0,1,0,0,0,1


In [2]:
del df["Unnamed: 0"]
df

,Survived,Age,firstclass,secondclass,thirdclass,C,Q,S,female,male
0,0,22.0,0,0,1,0,0,1,0,1
1,1,38.0,1,0,0,1,0,0,1,0
2,1,26.0,0,0,1,0,0,1,1,0
3,1,35.0,1,0,0,0,0,1,1,0
4,0,35.0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
709,0,39.0,0,0,1,0,1,0,1,0
710,0,27.0,0,1,0,0,0,1,0,1
711,1,19.0,1,0,0,0,0,1,1,0
712,1,26.0,1,0,0,1,0,0,0,1


In [3]:
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Logistic regression with XGB

In [4]:
x = df.drop(columns="Survived")
y = df["Survived"]

In [13]:
dmatrix = xgb.DMatrix(x,y)

In [14]:
params = {"objective":"reg:logistic","max_depth":3}

In [47]:
titanic_cv = xgb.cv(dtrain= dmatrix,
                   params=params,
                    nfold=3,
                    num_boost_round=5,
                    metrics="error",
                    as_pandas=True,
                    seed=13)

In [48]:
display(titanic_cv)

,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.182073,0.009754,0.198880,0.010481
1,0.177871,0.005240,0.194678,0.012988
2,0.177171,0.005514,0.196079,0.014283
3,0.176471,0.004538,0.193277,0.014954
4,0.176471,0.005146,0.191877,0.013865


In [49]:
print('Doğruluk Değeri : {:.3f}'.format((1-titanic_cv["test-error-mean"]).max()))

Doğruluk Değeri : 0.808


# Linear Regression with XGB

In [51]:
x=pd.read_csv("house_features_for_model.csv")
y=pd.read_csv("house_prices_for_model.csv")

In [84]:
from sklearn.model_selection import train_test_split
xtr,xte,ytr,yte= train_test_split(x, y, test_size=0.2, random_state=123)

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', seed=123)

In [85]:
import numpy as np
from sklearn.metrics import mean_squared_error

xg_reg.fit(xtr,ytr)

yhat=xg_reg.predict(xte)
rmse = np.sqrt(mean_squared_error(yte,yhat))

print("RMSE: %f" % (rmse))

RMSE: 36793.789932


In [86]:
DM_train = xgb.DMatrix(data = xtr, label=ytr)
DM_test =  xgb.DMatrix(data = xte, label=yte)
params = {"booster":"gblinear", "objective":"reg:squarederror"}
xg_reg = xgb.train(dtrain=dmatrix, params=params, num_boost_round=5)

In [87]:
yhat_gblinear = xg_reg.predict(DM_test)
rmse = np.sqrt(mean_squared_error(yte, yhat_gblinear))
print("RMSE: %f" % (rmse))

RMSE: 60525.303479


# Linear Regressionn with Cross validation on XGB

In [88]:
params = {"objective":"reg:squarederror", "max_depth":4, "silent":1}

xg_reg_cv = xgb.cv(dtrain=dmatrix, params=params, nfold=4, 
                    num_boost_round=100, early_stopping_rounds = 5, metrics="rmse", as_pandas=True, seed=123)

display(xg_reg_cv.sort_values(by='test-rmse-mean').head(5))

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
14,23863.149902,575.669489,36524.798828,1826.971779
13,24230.291015,569.039887,36576.751954,1882.043701
12,24645.121094,568.661396,36790.885742,1961.456884
11,25256.978515,580.038186,36954.645508,2346.281242
10,25972.173828,620.219570,37108.987305,2301.314502


In [89]:
np.mean(xg_reg_cv["test-rmse-mean"])

55352.98821626667

# XGB with Regularization

In [100]:
l1_params = np.arange(0.01, 0.2, 0.01)
params = {"objective":"reg:linear","max_depth":3, "silent":1}

rmses_l1 = []

for alpha in l1_params:
    params["alpha"] = alpha
    cv_rmse = xgb.cv(dtrain=dmatrix, params=params, nfold=4, num_boost_round=100, 
                             metrics="rmse", as_pandas=True, early_stopping_rounds=10, seed=123)
    
    rmses_l1.append(cv_rmse["test-rmse-mean"].tail(1).values[0])

In [101]:
print("Best L1 values:")
en_iyi_degerler = pd.DataFrame(list(zip(l1_params, rmses_l1)), columns=["l1", "rmse"])
display(en_iyi_degerler.sort_values('rmse').head())

Best L1 values:


,l1,rmse
16,0.17,34986.800781
14,0.15,34986.801758
13,0.14,34986.802734
9,0.10,34986.802735
15,0.16,34986.802735


In [106]:
params["alpha"] = 0.17
cv_rmse = xgb.cv(dtrain=dmatrix, params=params, nfold=4, num_boost_round=100, 
                     metrics="rmse", as_pandas=True, early_stopping_rounds=10, seed=123)

cv_rmse.sort_values(by='test-rmse-mean').head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
69,19729.783203,358.176420,34986.800781,1050.955593
67,19895.646485,317.509750,35001.475586,1014.622558
66,19985.541992,357.703538,35013.032227,962.806781
68,19827.599610,337.772809,35013.895508,1048.960361
65,20070.319336,357.613620,35025.789062,988.371485


# Hyperparameters

In [222]:
en_iyi_parametreler = {'objective':'reg:squarederror',
                       'colsample_bytree': 0.7, 
                       'learning_rate': 0.5, 
                       'max_depth': 3, 
                       'min_child_weight': 3, 
                      }



In [223]:
cv_rmse = xgb.cv(dtrain=dmatrix, params=en_iyi_parametreler, nfold=4, num_boost_round=100, 
                     metrics="rmse", as_pandas=True, early_stopping_rounds=10, seed=123)

print("Mean Rmse on Test Set:",np.mean(cv_rmse["test-rmse-mean"]))
cv_rmse.sort_values(by='test-rmse-mean').head()


Mean Rmse on Test Set: 39971.974168491935


,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
30,22663.083496,387.298581,35291.738282,1401.911116
29,22797.826172,373.301208,35313.959961,1459.549580
26,23434.066406,383.556504,35355.044922,1462.474208
25,23695.524903,487.769500,35379.018555,1350.073273
24,23860.149414,420.174410,35406.958985,1362.964359
